# Title 

Continuous Assessment 1 06/10/2023 - BD & ADA Module // MSc in Data Analytics Y1 S2 - <b>Student ID 2023063

## Introduction
write intro

--

Data source: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop/data

## Practical Big Data (PySparkSQL) - Prep data

In [1]:
#Import PySparkSQL
import pyspark
from pyspark.sql import SparkSession        

# Create SparkSession
spark = (SparkSession
  .builder
  .appName("SparkSQL")
  .getOrCreate())

# Path to dataset
csv_file = "file:////home/hduser/Downloads/CA1/2020-Jan.csv"

In [2]:
# Read and create a temporary view
df = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(csv_file))
df.createOrReplaceTempView("rawdata")

In [3]:
# Visualise inferred schema
data = spark.sql("SELECT * FROM rawdata")
data.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [4]:
# Display 5 first rows
data.show(5)

+--------------------+----------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|          event_time|event_type|product_id|        category_id|category_code|   brand|price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|2020-01-01 00:00:...|      view|   5809910|1602943681873052386|         null| grattol| 5.24|595414620|4adb70bb-edbd-498...|
|2020-01-01 00:00:...|      view|   5812943|1487580012121948301|         null|kinetics| 3.97|595414640|c8c5205d-be43-4f1...|
|2020-01-01 00:00:...|      view|   5798924|1783999068867920626|         null|  zinger| 3.97|595412617|46a5010f-bd69-4fb...|
|2020-01-01 00:00:...|      view|   5793052|1487580005754995573|         null|    null| 4.92|420652863|546f6af3-a517-475...|
|2020-01-01 00:00:...|      view|   5899926|2115334439910245200|         null|    null| 3.92|484071203|cff70ddf-529e-4b0...|


In [5]:
# Number of users
action = spark.sql("SELECT count(distinct user_id) FROM rawdata")
action.show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                 410073|
+-----------------------+



In [6]:
# Number of events
action = spark.sql("SELECT count(distinct event_type) FROM rawdata")
action.show()

+--------------------------+
|count(DISTINCT event_type)|
+--------------------------+
|                         4|
+--------------------------+



In [7]:
# Number of products
action = spark.sql("SELECT count(distinct product_id) FROM rawdata")
action.show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                     45484|
+--------------------------+



In [8]:
# Top seller product
countProduct = spark.sql("SELECT product_id, count(*) FROM rawdata GROUP BY product_id ORDER BY count(*) desc")
countProduct.show(5)

+----------+--------+
|product_id|count(1)|
+----------+--------+
|   5809910|   36967|
|   5809912|   14310|
|   5917178|   12634|
|   5700037|   11429|
|   5686925|   11098|
+----------+--------+
only showing top 5 rows



In [9]:
# New dataframe with topseller product
topseller = spark.sql("SELECT DISTINCT * FROM rawdata WHERE product_id LIKE 5809910")
topseller.show(5)

+--------------------+----------------+----------+-------------------+-------------+-------+-----+---------+--------------------+
|          event_time|      event_type|product_id|        category_id|category_code|  brand|price|  user_id|        user_session|
+--------------------+----------------+----------+-------------------+-------------+-------+-----+---------+--------------------+
|2020-01-01 00:00:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|595414620|4adb70bb-edbd-498...|
|2020-01-01 14:21:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|592285428|33ecf984-0438-459...|
|2020-01-01 16:56:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|595651372|121148f9-4e42-48b...|
|2020-01-01 18:40:...|remove_from_cart|   5809910|1602943681873052386|         null|grattol| 5.24|583351843|85fa8bff-02e8-4a5...|
|2020-01-02 06:09:...|            view|   5809910|1602943681873052386|         null|gratto

In [10]:
# Drop columns
topseller = topseller.drop(*["product_id","category_id","category_code","brand","price"])

In [11]:
# Show updated topseller df
topseller.show(10,False)

+-----------------------+----------------+---------+------------------------------------+
|event_time             |event_type      |user_id  |user_session                        |
+-----------------------+----------------+---------+------------------------------------+
|2020-01-01 00:00:00 UTC|view            |595414620|4adb70bb-edbd-4981-b60f-a05bfd32683a|
|2020-01-01 14:21:02 UTC|view            |592285428|33ecf984-0438-459c-aea6-241319336d75|
|2020-01-01 16:56:51 UTC|view            |595651372|121148f9-4e42-48ba-94eb-7aca8935940b|
|2020-01-01 18:40:56 UTC|remove_from_cart|583351843|85fa8bff-02e8-4a5f-80b8-a129ce5555e3|
|2020-01-02 06:09:01 UTC|view            |529378930|b99ea2eb-1c67-4f39-ab20-792ebfb87f93|
|2020-01-02 08:21:31 UTC|view            |595842440|b257db78-93e8-46d4-9893-56194751a4d9|
|2020-01-03 06:16:01 UTC|view            |595798798|8b2d9742-eb2f-4bbd-8387-ace81cfdef2a|
|2020-01-03 11:17:22 UTC|cart            |596055322|74521846-45da-4a1e-a61e-19fbe7f6fb82|
|2020-01-0

In [12]:
# Customer behaviour before purchasing example
# Random customer collected from filtering event_type: 'purchase' 
from pyspark.sql.functions import asc
topseller.select("event_time","event_type","user_session").filter("user_id = 431397250").orderBy(asc("event_time")).show(truncate=False)

+-----------------------+----------+------------------------------------+
|event_time             |event_type|user_session                        |
+-----------------------+----------+------------------------------------+
|2020-01-07 14:33:37 UTC|view      |d6151353-322f-4a7a-b3a4-0c4c0e52e2f5|
|2020-01-07 14:39:42 UTC|cart      |d6151353-322f-4a7a-b3a4-0c4c0e52e2f5|
|2020-01-07 18:05:19 UTC|purchase  |697d43d2-07f9-4c59-aa7c-8dca123cebc0|
+-----------------------+----------+------------------------------------+



In [13]:
# Customer behaviour before purchasing example
# Random customer collected from filtering event_type: 'purchase' 
from pyspark.sql.functions import asc
topseller.select("event_time","event_type","user_session").filter("user_id = 46383252").orderBy(asc("event_time")).show(30,truncate=False)

+-----------------------+----------+------------------------------------+
|event_time             |event_type|user_session                        |
+-----------------------+----------+------------------------------------+
|2020-01-28 06:12:26 UTC|cart      |5549dfe6-109e-497f-a9f2-01c0aecdd73f|
|2020-01-28 06:14:08 UTC|view      |5549dfe6-109e-497f-a9f2-01c0aecdd73f|
|2020-01-28 06:17:17 UTC|view      |5549dfe6-109e-497f-a9f2-01c0aecdd73f|
|2020-01-28 06:53:25 UTC|purchase  |67347fb6-2e23-4695-8d32-03829cc6fdaf|
+-----------------------+----------+------------------------------------+



In [14]:
# Sort ASC topseller df by user_id and event 
sorted_by_user_event = topseller.orderBy(["user_id","event_type"], ascending= [True, True])

In [15]:
# Return result
sorted_by_user_event.show(5,truncate=False)

+-----------------------+----------------+--------+------------------------------------+
|event_time             |event_type      |user_id |user_session                        |
+-----------------------+----------------+--------+------------------------------------+
|2020-01-28 05:43:43 UTC|view            |4661182 |ce380fb8-bfdd-49a5-949f-eeac2c349af1|
|2020-01-28 05:41:23 UTC|view            |4661182 |ce380fb8-bfdd-49a5-949f-eeac2c349af1|
|2020-01-14 09:33:54 UTC|cart            |12961898|6cb94e2c-4db2-46ac-8c11-1ced638112bc|
|2020-01-13 11:36:50 UTC|cart            |12961898|91ceb8ab-fa0f-4929-b02e-9f36a19e73a5|
|2020-01-27 09:37:44 UTC|remove_from_cart|12961898|ea56a517-171d-4740-a9f7-f628f40274c8|
+-----------------------+----------------+--------+------------------------------------+
only showing top 5 rows



In [16]:
# Group df by user_id and event_type
event_type_count_user = sorted_by_user_event.groupby('user_id','event_type').count()

In [17]:
# New df with organised data counting event_type by user_id
sorted_event_type_count_user = event_type_count_user.orderBy(["user_id","event_type"], ascending= [True, True])

In [18]:
# Display new df
sorted_event_type_count_user.show(10)

+--------+----------------+-----+
| user_id|      event_type|count|
+--------+----------------+-----+
| 4661182|            view|    2|
|12961898|            cart|    2|
|12961898|remove_from_cart|    1|
|27692594|            cart|    1|
|31272873|            cart|    1|
|31272873|remove_from_cart|    1|
|46383252|            cart|    1|
|46383252|        purchase|    1|
|46383252|            view|    2|
|47076222|            view|    5|
+--------+----------------+-----+
only showing top 10 rows



In [19]:
# Convert PySpark Dataframe to Pandas
pandas_df = sorted_event_type_count_user.toPandas()
pandas_df.head()

,user_id,event_type,count
0,4661182,view,2
1,12961898,cart,2
2,12961898,remove_from_cart,1
3,27692594,cart,1
4,31272873,cart,1


In [20]:
# Rearrange column event_type using function .pivot() from pandas lib
pandas_df = pandas_df.pivot(index='user_id',columns='event_type',values='count')
pandas_df

event_type,cart,purchase,remove_from_cart,view
user_id,,,,
4661182,NaN,NaN,NaN,2.0
12961898,2.0,NaN,1.0,NaN
27692594,1.0,NaN,NaN,NaN
31272873,1.0,NaN,1.0,NaN
46383252,1.0,1.0,NaN,2.0
...,...,...,...,...
608796694,NaN,NaN,NaN,1.0
608802103,NaN,NaN,NaN,1.0
608804936,NaN,NaN,NaN,1.0


In [21]:
# Rearrange column position
pandas_df = pandas_df.reindex(columns=['view','cart','remove_from_cart','purchase'])
pandas_df

event_type,view,cart,remove_from_cart,purchase
user_id,,,,
4661182,2.0,NaN,NaN,NaN
12961898,NaN,2.0,1.0,NaN
27692594,NaN,1.0,NaN,NaN
31272873,NaN,1.0,1.0,NaN
46383252,2.0,1.0,NaN,1.0
...,...,...,...,...
608796694,1.0,NaN,NaN,NaN
608802103,1.0,NaN,NaN,NaN
608804936,1.0,NaN,NaN,NaN


In [22]:
# Export final df to csv
pandas_df.to_csv('customer_data.csv')

# Part 2 - Neural Network Model

In [23]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [24]:
# Load dataset
data = pd.read_csv("customer_data.csv")
data

,user_id,view,cart,remove_from_cart,purchase
0,4661182,2.0,NaN,NaN,NaN
1,12961898,NaN,2.0,1.0,NaN
2,27692594,NaN,1.0,NaN,NaN
3,31272873,NaN,1.0,1.0,NaN
4,46383252,2.0,1.0,NaN,1.0
...,...,...,...,...,...
18131,608796694,1.0,NaN,NaN,NaN
18132,608802103,1.0,NaN,NaN,NaN
18133,608804936,1.0,NaN,NaN,NaN
18134,608816772,4.0,NaN,NaN,NaN


In [25]:
# Replace NaN values with 0
data = data.fillna(0)

In [26]:
# Separate features and target variable
X = data[['view', 'cart', 'remove_from_cart']]
y = data[['purchase']]

In [27]:
# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [28]:
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
# Build neural network (NN) model 
nnmodel = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1) #output layer, no activation function
])

In [31]:
# Compile NN model

nnmodel.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [32]:
# Train NN model
nnmodel.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
363/363 [==============================] - 1s 915us/step - loss: 0.1026 - mae: 0.1628 - val_loss: 0.0927 - val_mae: 0.1632
Epoch 2/50
363/363 [==============================] - 0s 763us/step - loss: 0.0889 - mae: 0.1536 - val_loss: 0.0865 - val_mae: 0.1444
Epoch 3/50
363/363 [==============================] - 0s 743us/step - loss: 0.0871 - mae: 0.1509 - val_loss: 0.0853 - val_mae: 0.1541
Epoch 4/50
363/363 [==============================] - 0s 712us/step - loss: 0.0854 - mae: 0.1496 - val_loss: 0.0832 - val_mae: 0.1489
Epoch 5/50
363/363 [==============================] - 0s 773us/step - loss: 0.0845 - mae: 0.1468 - val_loss: 0.0833 - val_mae: 0.1474
Epoch 6/50
363/363 [==============================] - 0s 720us/step - loss: 0.0843 - mae: 0.1462 - val_loss: 0.0831 - val_mae: 0.1524
Epoch 7/50
363/363 [==============================] - 0s 832us/step - loss: 0.0845 - mae: 0.1470 - val_loss: 0.0840 - val_mae: 0.1427
Epoch 8/50
363/363 [==============================] - 0s 729us

In [33]:
# Evaluate NN model
loss, mae = nnmodel.evaluate(X_test, y_test)
print(f'Mean absolute error (MAE) on test data: {mae}')

114/114 [==============================] - 0s 475us/step - loss: 0.0729 - mae: 0.1380
Mean absolute error (MAE) on test data: 0.1380244493484497


In [34]:
# Make prediction
new_customer = np.array([[8,4,2]])
scaled_data = scaler.transform(new_customer)
prediction = nnmodel.predict(scaled_data)

print(f'Predicted number of purchases: {prediction[0][0]}')

1/1 [==============================] - 0s 31ms/step
Predicted number of purchases: 1.0471512079238892


In [35]:
# Hyperparameter tuning

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import Adam
from itertools import product

In [36]:
# Define the model as a function 
def create_model(learning_rate=0.01, num_hidden_layers=2, num_neurons=32, activation='relu'):
    model = Sequential()
    model.add(Dense(num_neurons, activation=activation, input_shape=(3,)))
    
    for _ in range(num_hidden_layers - 1):
        model.add(Dense(num_neurons, activation=activation))
    
    model.add(Dense(1)) #output layer, no activation function
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    
    return model

In [37]:
# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
# Define hyperparameters
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_hidden_layers': [1, 2, 3],
    'num_neurons': [16, 32, 64],
    'activation': ['relu', 'tanh']
}

In [39]:
# Perform the grid search
best_params = None
best_score = float('inf')

In [40]:
for learning_rate, num_hidden_layers, num_neurons, activation in product(
    param_grid['learning_rate'],
    param_grid['num_hidden_layers'],
    param_grid['num_neurons'],
    param_grid['activation']
):
    params = {
        'learning_rate': learning_rate,
        'num_hidden_layers': num_hidden_layers,
        'num_neurons': num_neurons,
        'activation': activation
    }

    # Create and compile the model
    model1 = create_model(**params)

    # Train the model on the training set
    model1.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Evaluate the model on the validation set
    _, mae = model1.evaluate(X_test, y_test)

    if mae < best_score:
        best_score = mae
        best_params = params

# Print the best hyperparameters and corresponding score
print("Best Hyperparameters: ", best_params)
print("Best MAE Score: ", best_score)

114/114 [==============================] - 0s 457us/step - loss: 0.2597 - mae: 0.4223
Best Hyperparameters:  {'learning_rate': 0.001, 'num_hidden_layers': 2, 'num_neurons': 64, 'activation': 'relu'}
Best MAE Score:  0.1315811276435852
